# Neural Networks

Neural networks can be constructed using the ``torch.nn`` package.


## 1.Neural Network Defination
A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or
  weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``

For example, look at this network that classifies digit images:

In [1]:
# Code below is a 5 layers CNN,including 2 convolution layer and 3 fully connected layer.   
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the ``forward`` function, and the ``backward``
function (where gradients are computed) is automatically defined for you
using ``autograd``.
You can use any of the Tensor operations in the ``forward`` function.

The learnable parameters of a model are returned by ``net.parameters()``



In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Let's try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0421, -0.0726,  0.0196,  0.0210, -0.0853, -0.0247, -0.0119,  0.0458,
         -0.0202,  0.1098]], grad_fn=<AddmmBackward0>)


Zero the gradient buffers of all parameters and backprops with random
gradients:

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## 2.Loss Function
A loss function takes the (output, target) pair of inputs, and computes a
value that estimates how far away the output is from the target.  
损失函数的输入是 (output, target) ，即网络输出和真实标签对的数据，然后返回一个数值表示网络输出和真实标签的差距。  
PyTorch 中其实已经定义了不少的损失函数(PyTorch 中其实已经定义了不少的损失函数()，这里仅采用简单的均方误差：nn.MSELoss ，例子如下：)，这里仅采用简单的均方误差：nn.MSELoss ，例子如下：

In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9592, grad_fn=<MseLossBackward0>)


这里，整个网络的数据输入到输出经历的计算图如下所示，其实也就是数据从输入层到输出层，计算 loss 的过程。

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d 

          -> view -> linear -> relu -> linear -> relu -> linear  
      
          -> MSELoss   
      
          -> loss 
如果调用 loss.backward() ，那么整个图都是可微分的，也就是说包括 loss ，图中的所有张量变量，只要其属性 requires_grad=True ，那么其梯度 .grad张量都会随着梯度一直累计。

用代码来说明：

In [8]:
# MSELoss
print(loss.grad_fn)
# Linear layer
print(loss.grad_fn.next_functions[0][0])
# Relu
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## 3.Backprop
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.

反向传播的实现只需要调用 ``loss.backward()`` 即可，当然首先需要清空当前梯度缓存，即``.zero_grad()`` 方法，否则之前的梯度会累加到当前的梯度，这样会影响权值参数的更新。

下面是一个简单的例子，以 ``conv1`` 层的偏置参数 ``bias`` 在反向传播前后的结果为例：



In [10]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0059,  0.0024, -0.0024, -0.0043,  0.0127, -0.0081])


## 4.Update the weights
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):  
采用随机梯度下降(Stochastic Gradient Descent, SGD)方法的最简单的更新权重规则如下：
.. code:: python

    weight = weight - learning_rate * gradient

We can implement this using simple Python code:  
按照这个规则，代码实现如下所示：  
.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different
update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
To enable this, we built a small package: ``torch.optim`` that
implements all these methods. Using it is very simple:  

但是这只是最简单的规则，深度学习有很多的优化算法，不仅仅是 ``SGD``，还有 ``Nesterov-SGD, Adam, RMSProp`` 等等，为了采用这些不同的方法，这里采用 ``torch.optim`` 库，使用例子如下所示：  


.. code:: python

    import torch.optim as optim

    # create your optimizer
    optimizer = optim.SGD(net.parameters(), lr=0.01)

    # in your training loop:
    optimizer.zero_grad()   # zero the gradient buffers 清空缓存
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()    # Does the update 更新权重


